## Writeup Template
### You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

---

**Vehicle Detection Project**

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
* Implement a sliding-window technique and to search for vehicles in images.
* Run the pipeline on a video stream and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.

[//]: # (Image References)
[image1]: ./example_car.png
[image2]: ./example_notcar.png
[image3]: ./example_boundedbox.png
[image4]: ./examples/sliding_window.jpg
[image5]: ./examples/bboxes_and_heat.png
[image6]: ./examples/labels_map.png
[image7]: ./examples/output_bboxes.png
[video1]: ./project_video.mp4



### Histogram of Oriented Gradients (HOG)

#### 1. Explain how (and identify where in your code) you extracted HOG/color features from the training images.

The code for this step is contained in the first code cell of the IPython notebook.

I started by reading in all the `vehicle` and `non-vehicle` images.  Here is an example of one of each of the `vehicle` and `non-vehicle` classes:

![alt text][image1]
(example of car)

![alt text][image2]
(example of not car)


I then employed a feature combination of spatial feature, histogram feature and hog_feature, which is defined as:
``` python
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        img_features.append(spatial_features)
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        img_features.append(hist_features)
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        img_features.append(hog_features)
```
From several trial and error, I reached to the best parameters as below for the model training:

```python
    color_space = 'YUV'
    orient = 11
    pix_per_cell = 16
    cell_per_block = 2
    hog_channel = 0
```


#### 2. Describe how (and identify where in your code) you trained a classifier using your selected HOG/color features (and color features if you used them).

I trained a linear SVM using test size of 0.2, as shown below:
```python

    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.2, random_state=rand_state)

    print('Using:',orient,'orientations',pix_per_cell,
        'pixels per cell and', cell_per_block,'cells per block')
    print('Feature vector length:', len(X_train[0]))
    # Use a linear SVC 
    svc = LinearSVC()
    # Check the training time for the SVC
    t=time.time()
    svc.fit(X_train, y_train)
```

I realized that using a lower C factor in the svc model would enforce bigger margin and higher tolerance. As a result, false positive drops significantly, as shown below. Another method I also introduced in this submission is the averaged heat map, which will be explained later.


```python
    svc = LinearSVC(C=0.01)

```

### Sliding Window Search

#### Describe how (and identify where in your code) you implemented a sliding window search.  How did you decide what scales to search and how much to overlap windows?

I decided to search windows with the size dependent on the position, so that I can skip the smaller sized windows on the unnecessary parts of the image (more close to camera, supposed to have bigger sized window for car). 

The code is as below:

```python
    for index in range(5):
        window_size = 200 - index * 25
        y_stop = 680 - index * 45
        windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[y_start, y_stop], 
                        xy_window=(window_size, window_size), xy_overlap=(overlap_ratio, overlap_ratio))

```


### Video Implementation

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (somewhat wobbly or unstable bounding boxes are ok as long as you are identifying the vehicles most of the time with minimal false positives.)
The video is uploaded to the github.

#### 2. Describe how (and identify where in your code) you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.

I recorded the positions of positive detections in each frame of the video.  From the positive detections I created a heatmap and then thresholded that map to identify vehicle positions.  I then used `scipy.ndimage.measurements.label()` to identify individual blobs in the heatmap.  I then assumed each blob corresponded to a vehicle.  I constructed bounding boxes to cover the area of each blob detected.  


```python
    stored_heat.append(heat)
    sum_heat = []
    sum_heat = heat * 0
    for jj in range(len(stored_heat)):
        sum_heat = sum_heat + stored_heat[jj]
    heat = sum_heat/len(stored_heat)
    heat = apply_threshold(heat,1)
```

Here's an example result showing the bounded boxes of car. I used the averaged heat map over the last 5 frames to further reduce the false positive detection, as below.



![alt text][image3]


---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

There are still some noise, false positives in the image, but the results are much better now with the two methods I implanted-- lower C factor in classifier training, and averaged heatmap.
